In [1]:
from pymonad.tools import curry 
import sys
import random
import itertools

# Pig simulation

## Strategies

$Score : \cal{N} \times \cal{N}$

$Strategy : Score \times \left\{Score\right\} \rightarrow bool$ 

### stick at n

In [2]:
@curry(3)
def stick_at_n(n,player_score,opponent_scores) :
    _,turn_score = player_score
    return True if turn_score < n else False
    

In [3]:
stick_at_20 = stick_at_n(20)
stick_at_20((71,20),{(89,0)})

False

## Dice

In [4]:
def dice_class(sides,seed) : # remember - a class is just function that returns an object
    def dice() : # the object produced by the class - remember, an object is just a function with memory
        nonlocal seed # the objects memory
        random.seed(seed)
        seed = random.randint(0,sys.maxsize) # note - python 3 specific
        dice = dice_class(sides,seed) # notice the reference to the enclosing function
        return dice,random.randint(1,sides)
    return dice
        

In [5]:
D6 = dice_class(6,0)
for i in range(10) :
    D6,score = D6()
    print(score)

4
6
2
4
6
3
3
1
1
2


## Turn



$Player : Strategy \times Score$

$Turn : Strategy \times Score \times \left\{Score\right\} \rightarrow Score$

In [6]:
@curry(5)
def pig_turn(dice,jeopardy_score,strategy,player_score,opponent_scores) :
    current_score = accumulated,turn = player_score
    while strategy(current_score,opponent_scores) :
        dice,roll = dice()
        if roll == jeopardy_score :
            return player_score
        turn += roll
        current_score = accumulated,turn
    return accumulated + turn,0
        
        

In [7]:
D6 = dice_class(6,0)
stick_at_10 = stick_at_n(10)
jeopardy_score = 1
turn = pig_turn(D6,jeopardy_score)

In [8]:
player_score = (0,0)
opponent_scores = {(0,0)}

for i in range(20) :
    print(player_score)
    player_score = turn(stick_at_10,player_score,opponent_scores)

(0, 0)
(10, 0)
(22, 0)
(34, 0)
(44, 0)
(56, 0)
(56, 0)
(56, 0)
(56, 0)
(56, 0)
(71, 0)
(84, 0)
(94, 0)
(106, 0)
(121, 0)
(135, 0)
(146, 0)
(159, 0)
(170, 0)
(180, 0)


## Round

$Win : Score \rightarrow Bool$

$Round : \left\{Strategy\right\} \rightarrow Strategy$ 

In [9]:
@curry(2)
def pig_win(target,score) :
    accumulated,turn = score
    return False if accumulated + turn < target else True
    

In [10]:
@curry(3)
def pig_round(win,turn,strategies) :
    players = {s : (0,0) for s in strategies}
    while True :
        for strategy in players :
            score = turn(strategy,players[strategy],{players[s] for s in strategies if s != strategy})
            if win(score) : return strategy
            players[strategy] = score

In [11]:
win = pig_win(50)
D6 = dice_class(6,0)
jeopardy_score = 1
turn = pig_turn(D6,jeopardy_score)
stick_at_10 = stick_at_n(10)
stick_at_15 = stick_at_n(15)
strategies = (stick_at_10,stick_at_15)

In [12]:
around = pig_round(win,turn)

In [13]:
around(strategies) == stick_at_15

True

## Competition

$Competition : \left\{Strategy\right\} \rightarrow (Strategy \rightarrow \cal{N})$

In [14]:
@curry(3)
def pig_competition(around,n,strategies) :
    results = {s : 0 for s in strategies}
    for i in range(n) :
        s = around(strategies)
        results[around(strategies)] += 1
    return results

In [15]:
competition = pig_competition(around,100)
results = competition(strategies)

In [16]:
tuple(results.values())

(47, 53)

## Tournament

$Tournament : \left\{Strategy\right\} \rightarrow (Strategy^{n} \rightarrow \cal{R}^{n})$

In [17]:
@curry(3)
def pig_tournament(nplayers,competition,strategies) :
    D = [competition(s) for s in itertools.permutations(strategies,r=nplayers)]
    return {tuple(d.keys()) : tuple(d.values()) for d in D}
    

In [18]:
tournament = pig_tournament(2,competition)

In [19]:
results = tournament(strategies)

In [20]:
results

{(<function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>): (42,
  58),
 (<function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>,
  <function pymonad.tools._curry_helper.<locals>._curry_internal(*arguments: List[Any])>): (87,
  13)}

## Working Examples

In [26]:
win = pig_win(50)
D6 = dice_class(6,0)
jeopardy_score = 1
turn = pig_turn(D6,jeopardy_score)
entrants = {n : stick_at_n(n) for n in range(10,20)}
strategies = set(entrants.values())
around = pig_round(win,turn)
competition = pig_competition(around,100)
tournament = pig_tournament(3,competition)
results = tournament(strategies)